In [17]:
# import dependencies
import pandas as pd 
import requests 
import gmaps
from config import g_key 

In [18]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City ID,City,Country,Date,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed
0,0,Saint-Philippe,RE,2022-04-04 02:21:58,-21.3585,55.7679,74.79,86,97,13.78
1,1,Busselton,AU,2022-04-04 02:21:33,-33.6500,115.3333,68.63,52,15,14.54
2,2,Oistins,BB,2022-04-04 02:25:44,13.0667,-59.5333,78.89,69,40,18.41
3,3,Margate,GB,2022-04-04 02:27:44,51.3813,1.3862,40.69,70,98,5.99
4,4,Alta Floresta,BR,2022-04-04 02:23:07,-9.8756,-56.0861,71.69,96,100,1.19


In [19]:
city_data_df.dtypes

City ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max_Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [25]:
# get the maximum temperature 
max_temp = city_data_df["Max_Temp"]
temps = [] 
for temp in max_temp: 
    temps.append(max(temp,0))

In [48]:
#configure gmaps to use your google API key
gmaps.configure(api_key = g_key)

#heatmap of the temperature 
#get the latitude and longtitude
locations = city_data_df[["Lat", "Lng"]]

#get the maximum temperature 
max_temp = city_data_df["Max_Temp"]

# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure(center=(30,31), zoom_level = 1.5)

# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heat_layer = gmaps.heatmap_layer(locations, weights = temps, dissipating = False, 
                                 max_intensity = 300, point_radius = 4)


# 5. Add the heatmap layer.
fig.add_layer(heat_layer)

# 6. Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [47]:
# heatmap of percent humidity 
locations = city_data_df[['Lat', "Lng"]]
humidity = city_data_df['Humidity']

fig = gmaps.figure(center=(30,31), zoom_level = 1.5)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, 
                         max_intensity = 300, point_radius =4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [50]:
# heatmap of percent cloudiness 
locations = city_data_df[['Lat', "Lng"]]
clouds = city_data_df['Cloudiness']

fig = gmaps.figure(center=(30,31), zoom_level = 1.5)

heat_layer = gmaps.heatmap_layer(locations, weights = clouds, dissipating = False, 
                         max_intensity = 300, point_radius =4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [51]:
# heatmap of percent cloudiness 
locations = city_data_df[['Lat', "Lng"]]
wind = city_data_df['Wind Speed']

fig = gmaps.figure(center=(30,31), zoom_level = 1.5)

heat_layer = gmaps.heatmap_layer(locations, weights = wind, dissipating = False, 
                         max_intensity = 300, point_radius =4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [53]:
# ask the customer to add a minimum and maximum temperature value 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [54]:
# filter the dataset to find the cities that fit the criteria. 
preferred_cities_df = city_data_df.loc[(city_data_df['Max_Temp']<= max_temp) & (city_data_df['Max_Temp']>= min_temp)]
preferred_cities_df.head()

,City ID,City,Country,Date,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed
2,2,Oistins,BB,2022-04-04 02:25:44,13.0667,-59.5333,78.89,69,40,18.41
6,6,Bandarbeyla,SO,2022-04-04 02:21:42,9.4942,50.8122,75.69,82,6,2.53
7,7,Georgetown,MY,2022-04-04 02:21:48,5.4112,100.3354,84.13,82,20,6.91
10,10,Santo Domingo,DO,2022-04-04 02:27:45,18.5001,-69.9886,79.03,85,40,1.99
11,11,Kinshasa,CD,2022-04-04 02:27:46,-4.3246,15.3215,75.58,94,100,1.14


In [55]:
preferred_cities_df.count()

City ID       149
City          149
Country       149
Date          149
Lat           149
Lng           149
Max_Temp      149
Humidity      149
Cloudiness    149
Wind Speed    149
dtype: int64

In [57]:
# create dataframe called hotel_df to store hotel  names along with city, country, max_temp, and coordinates 
hotel_df = preferred_cities_df[['City', 'Country', 'Max_Temp', "Lat", 'Lng']].copy()
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Max_Temp,Lat,Lng,Hotel Name
2,Oistins,BB,78.89,13.0667,-59.5333,
6,Bandarbeyla,SO,75.69,9.4942,50.8122,
7,Georgetown,MY,84.13,5.4112,100.3354,
10,Santo Domingo,DO,79.03,18.5001,-69.9886,
11,Kinshasa,CD,75.58,-4.3246,15.3215,


In [79]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key, 
}

info_box_template = """
    <dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Max Temp</dt><dd>{Max_Temp} °F</dd>
    </dl>
    """

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [72]:
# iterate through the dataframe 

for index, row in hotel_df.iterrows(): 
    
    #get the latitude and longtitude 
    lat = row['Lat']
    lng = row['Lng']
    
    #add the latitude and longtitude to location key for the params dictionary 
    params['location'] = f"{lat},{lng}"
    
    #use the search term 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #make request and get the JSON data from the research 
    hotels = requests.get(base_url, params = params).json()
    
   
    #grab the first hotel from the results and store the name 
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except(IndexError): 
        print("Hotel not found... skipping.")

hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max_Temp,Lat,Lng,Hotel Name
2,Oistins,BB,78.89,13.0667,-59.5333,Butterfly Beach Hotel
6,Bandarbeyla,SO,75.69,9.4942,50.8122,JABIR HOTEL
7,Georgetown,MY,84.13,5.4112,100.3354,Cititel Penang
10,Santo Domingo,DO,79.03,18.5001,-69.9886,Cabañas City Heart
11,Kinshasa,CD,75.58,-4.3246,15.3215,Star Hotel II


In [80]:
# add a heatmap of temperature for the vacation spots 
locations = hotel_df[['Lat','Lng']]
max_temp = hotel_df['Max_Temp']

fig = gmaps.figure(center=(30.0,31.0), zoom_level = 1.5)

heat_layer = gmaps.heatmap_layer(locations, weights = max_temp, dissipating = False, 
                                max_intensity = 300, point_radius = 4)

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))